# Here i use median for fill missing value get 80.4% and the model is Xgboost

In [ ]:
!pip install xgboost

In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
import xgboost

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
from xgboost import XGBClassifier
# Import LightGBM
from lightgbm import LGBMClassifier

# Import CatBoost
from catboost import CatBoostClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/Dataset/Obesity Level Prediction-Fall 2024/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Dataset/Obesity Level Prediction-Fall 2024/test.csv')

In [ ]:
sample_submission = pd.read_csv('/content/drive/MyDrive/Dataset/Obesity Level Prediction-Fall 2024/sample_submission.csv',usecols=['NObeyesdad'])
sample_submission

,NObeyesdad
0,Normal_Weight
1,Normal_Weight
2,Normal_Weight
3,Overweight_Level_I
4,Overweight_Level_II
...,...
996,Overweight_Level_II
997,Overweight_Level_II
998,Overweight_Level_II
999,Overweight_Level_II


In [ ]:
train_data.head()

,ID,Age,Gender,Height,Weight,CALC,FAVC,FCVC,NCP,SCC,SMOKE,CH2O,family_history_with_overweight,FAF,TUE,CAEC,MTRANS,NObeyesdad
0,1,21.0,Female,1.62,64.0,no,no,2.0,3.0,no,no,2.0,yes,0.0,1.0,Sometimes,Public_Transportation,Normal_Weight
1,2,21.0,NaN,1.52,56.0,Sometimes,no,3.0,3.0,yes,yes,3.0,yes,3.0,NaN,Sometimes,Public_Transportation,Normal_Weight
2,3,NaN,Male,NaN,77.0,Frequently,no,2.0,3.0,no,no,2.0,yes,2.0,NaN,Sometimes,Public_Transportation,Normal_Weight
3,4,27.0,NaN,NaN,87.0,Frequently,no,3.0,3.0,no,NaN,2.0,NaN,2.0,0.0,NaN,Walking,Overweight_Level_I
4,5,22.0,Male,1.78,89.8,Sometimes,no,2.0,1.0,no,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [ ]:
train_data['FAVC'].unique()

array(['no', 'yes'], dtype=object)

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1110 entries, 0 to 1109
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              1110 non-null   int64  
 1   Age                             801 non-null    float64
 2   Gender                          760 non-null    object 
 3   Height                          873 non-null    float64
 4   Weight                          753 non-null    float64
 5   CALC                            1056 non-null   object 
 6   FAVC                            609 non-null    object 
 7   FCVC                            1110 non-null   float64
 8   NCP                             1108 non-null   float64
 9   SCC                             1110 non-null   object 
 10  SMOKE                           777 non-null    object 
 11  CH2O                            1110 non-null   float64
 12  family_history_with_overweight  99

In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              1001 non-null   int64  
 1   Age                             721 non-null    float64
 2   Gender                          662 non-null    object 
 3   Height                          781 non-null    float64
 4   Weight                          684 non-null    float64
 5   CALC                            933 non-null    object 
 6   FAVC                            1001 non-null   object 
 7   FCVC                            859 non-null    float64
 8   NCP                             1001 non-null   float64
 9   SCC                             668 non-null    object 
 10  SMOKE                           1001 non-null   object 
 11  CH2O                            916 non-null    float64
 12  family_history_with_overweight  87

In [ ]:
train_data.isnull().mean()*100

,0
ID,0.000000
Age,27.837838
Gender,31.531532
Height,21.351351
Weight,32.162162
CALC,4.864865
FAVC,45.135135
FCVC,0.000000
NCP,0.180180
SCC,0.000000


In [ ]:
test_data.isnull().mean()*100

,0
ID,0.000000
Age,27.972028
Gender,33.866134
Height,21.978022
Weight,31.668332
CALC,6.793207
FAVC,0.000000
FCVC,14.185814
NCP,0.000000
SCC,33.266733


In [ ]:
numeric_features = train_data.select_dtypes(include=['float64', 'int64']).columns.drop('ID')
categorical_features = train_data.select_dtypes(include=['object']).columns.drop('NObeyesdad')

numeric_imputer = KNNImputer(n_neighbors=10,weights='distance',add_indicator=True)
train_data[numeric_features] = numeric_imputer.fit_transform(train_data[numeric_features])
test_data[numeric_features] = numeric_imputer.transform(test_data[numeric_features])

categorical_imputer = SimpleImputer(strategy='most_frequent')
train_data[categorical_features] = categorical_imputer.fit_transform(train_data[categorical_features])
test_data[categorical_features] = categorical_imputer.transform(test_data[categorical_features])

In [ ]:
train_data.isnull().sum().sum()

0

In [ ]:
train_data

,ID,Age,Gender,Height,Weight,CALC,FAVC,FCVC,NCP,SCC,SMOKE,CH2O,family_history_with_overweight,FAF,TUE,CAEC,MTRANS,NObeyesdad
0,1,21.000000,Female,1.620000,64.000000,no,no,2.000000,3.000000,no,no,2.000000,yes,0.000000,1.000000,Sometimes,Public_Transportation,Normal_Weight
1,2,21.000000,Male,1.520000,56.000000,Sometimes,no,3.000000,3.000000,yes,yes,3.000000,yes,3.000000,0.773087,Sometimes,Public_Transportation,Normal_Weight
2,3,21.000000,Male,1.691303,77.000000,Frequently,no,2.000000,3.000000,no,no,2.000000,yes,2.000000,0.773087,Sometimes,Public_Transportation,Normal_Weight
3,4,27.000000,Male,1.691303,87.000000,Frequently,no,3.000000,3.000000,no,no,2.000000,yes,2.000000,0.000000,Sometimes,Walking,Overweight_Level_I
4,5,22.000000,Male,1.780000,89.800000,Sometimes,no,2.000000,1.000000,no,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1105,1106,21.000000,Male,1.823765,96.945262,Sometimes,yes,1.588782,2.601675,no,no,2.469469,yes,1.736538,1.886855,Sometimes,Public_Transportation,Overweight_Level_II
1106,1107,33.049121,Male,1.691303,83.001642,no,yes,2.000000,1.171027,no,no,2.405172,yes,2.300282,0.000000,Sometimes,Automobile,Overweight_Level_II
1107,1108,37.205173,Female,1.667469,80.993373,no,yes,2.010540,2.776840,no,no,1.651548,yes,0.000000,0.773087,Sometimes,Automobile,Overweight_Level_II
1108,1109,19.027417,Female,1.588147,67.722222,Sometimes,no,3.000000,3.362758,no,no,2.000000,yes,2.892922,1.000000,Sometimes,Public_Transportation,Overweight_Level_II


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1110 entries, 0 to 1109
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              1110 non-null   int64  
 1   Age                             1110 non-null   float64
 2   Gender                          1110 non-null   object 
 3   Height                          1110 non-null   float64
 4   Weight                          1110 non-null   float64
 5   CALC                            1110 non-null   object 
 6   FAVC                            1110 non-null   object 
 7   FCVC                            1110 non-null   float64
 8   NCP                             1110 non-null   float64
 9   SCC                             1110 non-null   object 
 10  SMOKE                           1110 non-null   object 
 11  CH2O                            1110 non-null   float64
 12  family_history_with_overweight  11

In [ ]:
test_data.isnull().sum().sum()

0

In [ ]:
encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded_train = pd.DataFrame(encoder.fit_transform(train_data[categorical_features]),
                             columns=encoder.get_feature_names_out(categorical_features))
encoded_test = pd.DataFrame(encoder.transform(test_data[categorical_features]),
                            columns=encoder.get_feature_names_out(categorical_features))



scaler = StandardScaler()
scaled_train = pd.DataFrame(scaler.fit_transform(train_data[numeric_features]),
                            columns=numeric_features)
scaled_test = pd.DataFrame(scaler.transform(test_data[numeric_features]),
                           columns=numeric_features)


processed_train = pd.concat([scaled_train, encoded_train], axis=1)
processed_test = pd.concat([scaled_test, encoded_test], axis=1)

processed_train['NObeyesdad'] = train_data['NObeyesdad']

In [ ]:
processed_train

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender_Male,CALC_Sometimes,...,SMOKE_yes,family_history_with_overweight_yes,CAEC_Frequently,CAEC_Sometimes,CAEC_no,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking,NObeyesdad
0,-0.246178,-0.793793,-0.280195,-0.617439,0.369099,0.073697,-1.230712,0.475331,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,Normal_Weight
1,-0.246178,-1.956425,-0.854089,1.200034,0.369099,1.658611,2.053809,0.098149,1.0,1.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,Normal_Weight
2,-0.246178,0.035199,0.652384,-0.617439,0.369099,0.073697,0.958969,0.098149,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,Normal_Weight
3,0.848088,0.035199,1.369753,1.200034,0.369099,0.073697,0.958969,-1.186898,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,Overweight_Level_I
4,-0.063801,1.066418,1.570616,-0.617439,-1.842506,0.073697,-1.230712,-1.186898,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1105,-0.246178,1.575244,2.083194,-1.364817,-0.071370,0.817765,0.670520,1.949487,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,Overweight_Level_II
1106,1.951312,0.035199,1.082923,-0.617439,-1.653384,0.715859,1.287730,-1.186898,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Overweight_Level_II
1107,2.709283,-0.241903,0.938856,-0.598283,0.122328,-0.478570,-1.230712,0.098149,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Overweight_Level_II
1108,-0.605933,-1.164126,-0.013174,1.200034,0.770238,0.073697,1.936576,0.475331,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,Overweight_Level_II


In [ ]:
processed_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1110 entries, 0 to 1109
Data columns (total 23 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Age                                 1110 non-null   float64
 1   Height                              1110 non-null   float64
 2   Weight                              1110 non-null   float64
 3   FCVC                                1110 non-null   float64
 4   NCP                                 1110 non-null   float64
 5   CH2O                                1110 non-null   float64
 6   FAF                                 1110 non-null   float64
 7   TUE                                 1110 non-null   float64
 8   Gender_Male                         1110 non-null   float64
 9   CALC_Sometimes                      1110 non-null   float64
 10  CALC_no                             1110 non-null   float64
 11  FAVC_yes                            1110 no

In [ ]:
processed_train

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender_Male,CALC_Sometimes,...,SMOKE_yes,family_history_with_overweight_yes,CAEC_Frequently,CAEC_Sometimes,CAEC_no,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking,NObeyesdad
0,-0.246178,-0.793793,-0.280195,-0.617439,0.369099,0.073697,-1.230712,0.475331,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,Normal_Weight
1,-0.246178,-1.956425,-0.854089,1.200034,0.369099,1.658611,2.053809,0.098149,1.0,1.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,Normal_Weight
2,-0.246178,0.035199,0.652384,-0.617439,0.369099,0.073697,0.958969,0.098149,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,Normal_Weight
3,0.848088,0.035199,1.369753,1.200034,0.369099,0.073697,0.958969,-1.186898,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,Overweight_Level_I
4,-0.063801,1.066418,1.570616,-0.617439,-1.842506,0.073697,-1.230712,-1.186898,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1105,-0.246178,1.575244,2.083194,-1.364817,-0.071370,0.817765,0.670520,1.949487,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,Overweight_Level_II
1106,1.951312,0.035199,1.082923,-0.617439,-1.653384,0.715859,1.287730,-1.186898,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Overweight_Level_II
1107,2.709283,-0.241903,0.938856,-0.598283,0.122328,-0.478570,-1.230712,0.098149,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Overweight_Level_II
1108,-0.605933,-1.164126,-0.013174,1.200034,0.770238,0.073697,1.936576,0.475331,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,Overweight_Level_II


In [ ]:
processed_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 22 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Age                                 1001 non-null   float64
 1   Height                              1001 non-null   float64
 2   Weight                              1001 non-null   float64
 3   FCVC                                1001 non-null   float64
 4   NCP                                 1001 non-null   float64
 5   CH2O                                1001 non-null   float64
 6   FAF                                 1001 non-null   float64
 7   TUE                                 1001 non-null   float64
 8   Gender_Male                         1001 non-null   float64
 9   CALC_Sometimes                      1001 non-null   float64
 10  CALC_no                             1001 non-null   float64
 11  FAVC_yes                            1001 no

# LogisticRegression

In [ ]:
X = processed_train.drop(columns='NObeyesdad')
y = train_data['NObeyesdad']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
Lr_model = LogisticRegression()
Lr_model.fit(X_train, y_train)
y_pred = Lr_model.predict(X_val)
accuracy_for_Lr = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy_for_Lr}')
train_accuracy = accuracy_score(y_train, Lr_model.predict(X_train))
print(f'Train Accuracy: {train_accuracy}')

Accuracy: 0.6306306306306306
Train Accuracy: 0.6756756756756757


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# DecisionTreeClassifier

In [ ]:
X = processed_train.drop(columns='NObeyesdad')
y = train_data['NObeyesdad']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)
y_pred = dt_model.predict(X_val)
accuracy_for_dt = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy_for_dt}')
train_accuracy = accuracy_score(y_train, dt_model.predict(X_train))
print(f'Train Accuracy: {train_accuracy}')

Accuracy: 0.6801801801801802
Train Accuracy: 1.0


# SVC

In [ ]:
X = processed_train.drop(columns='NObeyesdad')
y = train_data['NObeyesdad']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
svm_model = SVC()
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_val)
accuracy_for_svm = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy_for_svm}')
  # Calculate and print the train accuracy
train_accuracy = accuracy_score(y_train, svm_model.predict(X_train))
print(f'Train Accuracy: {train_accuracy}')

Accuracy: 0.6756756756756757
Train Accuracy: 0.7984234234234234


# KNeighborsClassifier

In [ ]:
X = processed_train.drop(columns='NObeyesdad')
y = train_data['NObeyesdad']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
knc_model = KNeighborsClassifier()
knc_model.fit(X_train, y_train)
y_pred = knc_model.predict(X_val)
accuracy_for_knc = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy_for_knc}')
train_accuracy = accuracy_score(y_train, knc_model.predict(X_train))
print(f'Train Accuracy: {train_accuracy}')

Accuracy: 0.6621621621621622
Train Accuracy: 0.7804054054054054


# Naive Bayes

In [ ]:
  X = processed_train.drop(columns='NObeyesdad')
  y = train_data['NObeyesdad']
  X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)
  nb_model = GaussianNB()
  nb_model.fit(X_train, y_train)
  y_pred = nb_model.predict(X_val)
  accuracy_for_nb = accuracy_score(y_val, y_pred)
  print(f'Accuracy: {accuracy_for_nb}')
  train_accuracy = accuracy_score(y_train, nb_model.predict(X_train))
  print(f'Train Accuracy: {train_accuracy}')

Accuracy: 0.11711711711711711
Train Accuracy: 0.12387387387387387


# MLPClassifier

In [ ]:
X=processed_train.drop(columns='NObeyesdad')
y=train_data['NObeyesdad']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)
mlp_model = MLPClassifier(hidden_layer_sizes=(100,50), activation='relu',max_iter=500,solver='adam',learning_rate_init=0.01,verbose=True, random_state=42)
mlp_model.fit(X_train, y_train)
y_pred = mlp_model.predict(X_val)
accuracy_for_mlp = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy_for_mlp}')
train_accuracy = accuracy_score(y_train, mlp_model.predict(X_train))
print(f'Train Accuracy: {train_accuracy}')

Iteration 1, loss = 1.66565904
Iteration 2, loss = 1.23115531
Iteration 3, loss = 1.06994856
Iteration 4, loss = 0.97246216
Iteration 5, loss = 0.88162932
Iteration 6, loss = 0.82672280
Iteration 7, loss = 0.75116147
Iteration 8, loss = 0.71349332
Iteration 9, loss = 0.67513439
Iteration 10, loss = 0.65999692
Iteration 11, loss = 0.62532865
Iteration 12, loss = 0.57365035
Iteration 13, loss = 0.54364121
Iteration 14, loss = 0.53076400
Iteration 15, loss = 0.47404886
Iteration 16, loss = 0.45176378
Iteration 17, loss = 0.42337577
Iteration 18, loss = 0.41722423
Iteration 19, loss = 0.41483403
Iteration 20, loss = 0.37979700
Iteration 21, loss = 0.37447920
Iteration 22, loss = 0.32618463
Iteration 23, loss = 0.31624280
Iteration 24, loss = 0.28665995
Iteration 25, loss = 0.26319285
Iteration 26, loss = 0.24550169
Iteration 27, loss = 0.23983952
Iteration 28, loss = 0.22362331
Iteration 29, loss = 0.21039752
Iteration 30, loss = 0.18514907
Iteration 31, loss = 0.18359833
Iteration 32, los

#Catboost

In [ ]:
X=processed_train.drop(columns='NObeyesdad')
y=train_data['NObeyesdad']



X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42,stratify=y)
cat_model = CatBoostClassifier()
cat_model.fit(X_train, y_train)
y_pred = cat_model.predict(X_val)
accuracy_for_cat = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy_for_cat}')
train_accuracy = accuracy_score(y_train, cat_model.predict(X_train))
print(f'Train Accuracy: {train_accuracy}')

Learning rate set to 0.078649
0:	learn: 1.8708421	total: 179ms	remaining: 2m 59s
1:	learn: 1.8023838	total: 231ms	remaining: 1m 55s
2:	learn: 1.7323773	total: 265ms	remaining: 1m 28s
3:	learn: 1.6624009	total: 301ms	remaining: 1m 14s
4:	learn: 1.6144640	total: 358ms	remaining: 1m 11s
5:	learn: 1.5583874	total: 392ms	remaining: 1m 4s
6:	learn: 1.5133994	total: 420ms	remaining: 59.5s
7:	learn: 1.4678270	total: 460ms	remaining: 57s
8:	learn: 1.4309550	total: 517ms	remaining: 56.9s
9:	learn: 1.3993888	total: 571ms	remaining: 56.6s
10:	learn: 1.3699300	total: 667ms	remaining: 60s
11:	learn: 1.3318525	total: 739ms	remaining: 1m
12:	learn: 1.3027789	total: 799ms	remaining: 1m
13:	learn: 1.2753501	total: 865ms	remaining: 1m
14:	learn: 1.2480279	total: 937ms	remaining: 1m 1s
15:	learn: 1.2230482	total: 978ms	remaining: 1m
16:	learn: 1.2020722	total: 1.03s	remaining: 59.8s
17:	learn: 1.1820702	total: 1.09s	remaining: 59.7s
18:	learn: 1.1617965	total: 1.15s	remaining: 59.3s
19:	learn: 1.1408728	t

# GradientBoostingClassifier

In [ ]:
X=processed_train.drop(columns='NObeyesdad')
y=train_data['NObeyesdad']



X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42,stratify=y)
gb_model = GradientBoostingClassifier()
gb_model.fit(X_train, y_train)
y_pred = gb_model.predict(X_val)
accuracy_for_gb = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy_for_gb}')
train_accuracy = accuracy_score(y_train, gb_model.predict(X_train))
print(f'Train Accuracy: {train_accuracy}')

Accuracy: 0.7792792792792793
Train Accuracy: 0.9774774774774775


# AdaBoostClassifier

In [ ]:
X=processed_train.drop(columns='NObeyesdad')
y=train_data['NObeyesdad']


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)
abc_model = AdaBoostClassifier()
abc_model.fit(X_train, y_train)
y_pred = abc_model.predict(X_val)
accuracy_for_abc = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy_for_abc}')
train_accuracy = accuracy_score(y_train, abc_model.predict(X_train))
print(f'Train Accuracy: {train_accuracy}')

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy: 0.3153153153153153
Train Accuracy: 0.3310810810810811


# LgbmClassifier

In [ ]:
X=processed_train.drop(columns='NObeyesdad')
y=train_data['NObeyesdad']



X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.21, random_state=42)
lgbm_model = LGBMClassifier()
lgbm_model.fit(X_train, y_train)
y_pred = lgbm_model.predict(X_val)
accuracy_for_lgbm = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy_for_lgbm}')
train_accuracy = accuracy_score(y_train, lgbm_model.predict(X_train))
print(f'Train Accuracy: {train_accuracy}')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000825 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1526
[LightGBM] [Info] Number of data points in the train set: 876, number of used features: 20
[LightGBM] [Info] Start training from score -1.414074
[LightGBM] [Info] Start training from score -1.390871
[LightGBM] [Info] Start training from score -3.137780
[LightGBM] [Info] Start training from score -4.472781
[LightGBM] [Info] Start training from score -4.472781
[LightGBM] [Info] Start training from score -1.328629
[LightGBM] [Info] Start training from score -1.731941
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

# RandomForestClassifier

In [ ]:
X = processed_train.drop(columns='NObeyesdad')
y = train_data['NObeyesdad']


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_val)
accuracy_for_rf = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy_for_rf}')
train_accuracy = accuracy_score(y_train, rf_model.predict(X_train))
print(f'Train Accuracy: {train_accuracy}')

Accuracy: 0.7927927927927928
Train Accuracy: 1.0


# XGBClassifier

In [ ]:
a = processed_train.drop(columns='NObeyesdad')
b = train_data['NObeyesdad']

# Create a LabelEncoder object
le = LabelEncoder()

# Fit the encoder to the target variable and transform it
b = le.fit_transform(b)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(a, b, test_size=0.23, random_state=42)

# Create and train the XGBClassifier model
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

# Make predictions on the validation set
y_pred = xgb_model.predict(X_val)

# Evaluate the model's accuracy
accuracy_for_xgb = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy_for_xgb}')
train_accuracy = accuracy_score(y_train, xgb_model.predict(X_train))
print(f'Train Accuracy: {train_accuracy}')

Accuracy: 0.796875
Train Accuracy: 1.0


55555555555555555555555555555555555555555555555555555555555555555555555555

# Now final test

In [ ]:
x=processed_train.drop(columns='NObeyesdad')
y=train_data['NObeyesdad']

# RandomForestClassifier

In [ ]:
from typing import final
final_rf_model = RandomForestClassifier(random_state=42)
final_rf_model.fit(x, y)

final_test_predictions_rf = final_rf_model.predict(processed_test)
final_test_predictions_rf

array(['Overweight_Level_II', 'Overweight_Level_II',
       'Overweight_Level_II', ..., 'Obesity_Type_III', 'Obesity_Type_III',
       'Obesity_Type_III'], dtype=object)

# KNeighborsClassifier

In [ ]:
final_knc_model = KNeighborsClassifier()
final_knc_model.fit(x,y)
final_test_predictions_knc = final_knc_model.predict(processed_test)
final_test_predictions_knc

array(['Overweight_Level_II', 'Overweight_Level_I', 'Overweight_Level_I',
       ..., 'Obesity_Type_III', 'Obesity_Type_III', 'Obesity_Type_III'],
      dtype=object)

# LgbmClassifier

In [ ]:
final_lgbm_model = LGBMClassifier()
final_lgbm_model.fit(x,y)
final_test_predictions_lgbm = final_lgbm_model.predict(processed_test)
final_test_predictions_lgbm

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000266 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1932
[LightGBM] [Info] Number of data points in the train set: 1110, number of used features: 20
[LightGBM] [Info] Start training from score -1.406313
[LightGBM] [Info] Start training from score -1.352633
[LightGBM] [Info] Start training from score -3.161968
[LightGBM] [Info] Start training from score -4.614220
[LightGBM] [Info] Start training from score -4.527209
[LightGBM] [Info] Start training from score -1.342234
[LightGBM] [Info] Start training from score -1.759842
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

array(['Overweight_Level_II', 'Overweight_Level_II',
       'Overweight_Level_II', ..., 'Obesity_Type_III', 'Obesity_Type_III',
       'Obesity_Type_III'], dtype=object)

# SVC

In [ ]:
final_svm_model = SVC()
final_svm_model.fit(x,y)
final_test_predictions_svm = final_svm_model.predict(processed_test)
final_test_predictions_svm

array(['Overweight_Level_II', 'Overweight_Level_II',
       'Overweight_Level_II', ..., 'Obesity_Type_III', 'Obesity_Type_III',
       'Obesity_Type_III'], dtype=object)

# DecisionTreeClassifier

In [ ]:
final_dt_model = DecisionTreeClassifier()
final_dt_model.fit(x,y)
final_test_predictions_dt = final_dt_model.predict(processed_test)
final_test_predictions_dt

array(['Overweight_Level_II', 'Overweight_Level_II',
       'Overweight_Level_II', ..., 'Obesity_Type_III', 'Obesity_Type_III',
       'Obesity_Type_III'], dtype=object)

# RandomForestClassifier

In [ ]:
final_rf_model = RandomForestClassifier(random_state=42)
final_rf_model.fit(x, y)

final_test_predictions_rf = final_rf_model.predict(processed_test)
final_test_predictions_rf

array(['Overweight_Level_II', 'Overweight_Level_II',
       'Overweight_Level_II', ..., 'Obesity_Type_III', 'Obesity_Type_III',
       'Obesity_Type_III'], dtype=object)

In [ ]:
submission = pd.DataFrame({
    'ID': test_data['ID'],
    'NObeyesdad': final_test_predictions_rf
})

submission.to_csv('submission_rf(median).csv', index=False)